In [12]:
import pandas as pd
dating = pd.read_csv("profiles.csv")

In [13]:
# basic data cleanup
# income
dating.loc[dating["income"] == -1, "income"] = pd.NA

# Let's create ML algorithm that will predict drug use on empty fields

prepare data some more for drug investigation

In [14]:
# create some suspicios words list that may relate to drug use to filter from essays
drug_words = ["party", "fun", "hang out", "festival", "good time", "drink"]
essay_cols = [col for col in dating.columns if col.startswith("essay")]
for word in drug_words:
    dating[word + "_count"] = dating[essay_cols].apply(
        lambda row: sum(str(cell).lower().count(word) for cell in row if pd.notna(cell)),
        axis=1
    )
    
dating = dating.drop(columns=essay_cols)

Prepare data for machine learning

In [15]:
drugs_values = dating["drugs"].value_counts()
dating_drugs_filled = dating.dropna(subset=["drugs"])
dating_drugs_missing = dating[dating["drugs"].isna()]
features = dating_drugs_filled.drop(columns=["drugs"])
labels = dating_drugs_filled["drugs"]
features["education"][features["education"].str.contains("graduated", na=False)].value_counts()
features["religion"].value_counts()
labels.value_counts(normalize=True)

drugs
never        0.822483
sometimes    0.168578
often        0.008939
Name: proportion, dtype: float64

In [16]:
# prepare features for ML
features = pd.get_dummies(features, columns=["body_type", "drinks", "job", "orientation", "pets"], dummy_na=False)


# bucket education column
def bucket_education(val):
    if pd.isna(val):
        return pd.NA
    val = str(val).lower()
    is_working_on = "working on" in val
    has_college_or_uni = any([exp in val for exp in ["college", "university", "masters program"]])
    has_phd_program = "ph.d program" in val
    has_graduated = "graduated" in val
    has_high_school = any(exp in val for exp in ["high school", "law school", "med school", "space camp"])
    has_dropped_out = "dropped out" in val

    if has_phd_program and not is_working_on and not has_dropped_out:
        return "Ph.D"

    if has_graduated and (has_college_or_uni or has_phd_program) or has_phd_program:
        return "graduated"

    if has_college_or_uni and not is_working_on and not has_dropped_out:
        return "graduated"

    if (has_high_school and not is_working_on and not has_dropped_out) or (
        (is_working_on or has_dropped_out) and has_college_or_uni
    ):
        return "high_school"

    return "lower"

# extract most common single ethnicities (about 5), rest as "other"
# literal list based on value_counts and inspection
ethnicities = [
    "white",
    "asian",
    "black",
    "hispanic / latin",
    "other",
    "middle eastern",
    "native american",
    "pacific islander",
    "indian"
]
for ethn in ethnicities:
    features[ethn] = features["ethnicity"].apply(lambda val: True if pd.notna(val) and ethn in val else False)

features["education_bucket"] = features["education"].apply(bucket_education)
features = pd.get_dummies(features, columns=["education_bucket"], dummy_na=False)
features["male"] = features["sex"].apply(lambda val: True if val == "m" else False)
features["smokes"] = features["smokes"].apply(lambda val: False if val == "no" else True)
features["is_single"] = features["status"].apply(lambda val: True if val == "single" else False)
features["has_kid_s"] = features["offspring"].apply(lambda val: True if pd.notna(val) and "has" in val else False)
def is_strict_diet(val):
    if pd.isna(val):
        return False

    has_specific_diet = not any(exp in val for exp in ["anything", "other"])
    return True if "strictly" in val and has_specific_diet else False

features["strict_diet"] = features["diet"].apply(is_strict_diet)
features["strict_religion"] = features["religion"].apply(lambda val: True if pd.notna(val) and "very serious about it" in val else False)
diets = ["vegan", "vegetarian", "halal", "kosher"]
for diet in diets:
    features[diet] = features["diet"].apply(lambda val: True if pd.notna(val) and diet in val else False)

features = features.select_dtypes(include=["number", "bool"])

In [17]:
education_unique = dating["education"].dropna().unique()
education_buckets = {edu: bucket_education(edu) for edu in education_unique}
education_buckets

{'working on college/university': 'high_school',
 'working on space camp': 'lower',
 'graduated from masters program': 'graduated',
 'graduated from college/university': 'graduated',
 'working on two-year college': 'high_school',
 'graduated from high school': 'high_school',
 'working on masters program': 'high_school',
 'graduated from space camp': 'high_school',
 'college/university': 'graduated',
 'dropped out of space camp': 'lower',
 'graduated from ph.d program': 'Ph.D',
 'graduated from law school': 'high_school',
 'working on ph.d program': 'graduated',
 'two-year college': 'graduated',
 'graduated from two-year college': 'graduated',
 'working on med school': 'lower',
 'dropped out of college/university': 'high_school',
 'space camp': 'high_school',
 'graduated from med school': 'high_school',
 'dropped out of high school': 'lower',
 'working on high school': 'lower',
 'masters program': 'graduated',
 'dropped out of ph.d program': 'graduated',
 'dropped out of two-year colleg

let's go with decision tree model

In [28]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import make_scorer, f1_score

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=.2, random_state=21)
dtree = DecisionTreeClassifier(class_weight="balanced", max_depth=10, random_state=10)
dtree.fit(x_train, y_train)
tree_text = tree.export_text(dtree, feature_names=list(x_train.columns))
print(tree_text)

|--- smokes <= 0.50
|   |--- income <= 375000.00
|   |   |--- orientation_bisexual <= 0.50
|   |   |   |--- drinks_often <= 0.50
|   |   |   |   |--- age <= 27.50
|   |   |   |   |   |--- education_bucket_graduated <= 0.50
|   |   |   |   |   |   |--- job_artistic / musical / writer <= 0.50
|   |   |   |   |   |   |   |--- drinks_socially <= 0.50
|   |   |   |   |   |   |   |   |--- education_bucket_high_school <= 0.50
|   |   |   |   |   |   |   |   |   |--- height <= 70.50
|   |   |   |   |   |   |   |   |   |   |--- class: never
|   |   |   |   |   |   |   |   |   |--- height >  70.50
|   |   |   |   |   |   |   |   |   |   |--- class: often
|   |   |   |   |   |   |   |   |--- education_bucket_high_school >  0.50
|   |   |   |   |   |   |   |   |   |--- drink_count <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- class: never
|   |   |   |   |   |   |   |   |   |--- drink_count >  0.50
|   |   |   |   |   |   |   |   |   |   |--- class: often
|   |   |   |   |   |   |   |--- dr

In [36]:
from sklearn.metrics import classification_report
scores = cross_val_score(dtree, x_test, y_test, scoring="f1_weighted", cv=5)
print("Macro-F1:", scores.mean())

# after fitting and predicting
y_pred = dtree.predict(x_test)
print(f1_score(y_test, y_pred, average="weighted"))

print(classification_report(
    y_test,
    y_pred,
    labels=['never','often','sometimes'],
    target_names=['never','often','sometimes'],
    zero_division=0         # avoid ‘undefined’ warnings if a class is missing
))

Macro-F1: 0.7376968172674336
0.7483778715815311
              precision    recall  f1-score   support

       never       0.91      0.76      0.83      7528
       often       0.05      0.42      0.09        83
   sometimes       0.35      0.49      0.41      1563

    accuracy                           0.71      9174
   macro avg       0.44      0.56      0.44      9174
weighted avg       0.81      0.71      0.75      9174

